In [1]:
from imgaug import augmentables
import imgaug.augmenters as iaa
import cv2
import glob
import json
import random
import matplotlib.pyplot as plt
from numpy.lib.function_base import interp
from numpy.lib.type_check import imag
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os
from tqdm import tqdm

In [9]:
def load_dataset(img_path, json_path):
    try:
        images_path = glob.glob(img_path)
        bounding_box_path = glob.glob(json_path)
    except:
        print('glob library 의 input 형식을 받아야합니다.')
        return None

    images = []
    for img_path in images_path:
        img = cv2.imread(img_path)
        images.append(img)

    bounding_box = []
    label_all = []
    for path in bounding_box_path:
        with open(path, encoding='utf-8') as json_file:
            json_data = json.load(json_file)

        temp = []
        temp_txt = []
        for i in json_data['shapes']:
            points = i['points']
            label = i['label']
            
            temp.append(BoundingBox(x1=points[0][0], y1= points[0][1], x2 = points[1][0], y2 = points[1][1]))
            temp_txt.append(label)

        label_all.append(temp_txt)
        bounding_box.append(temp)

    return images, label_all, bounding_box

In [10]:
images, label_all, bounding_Box = load_dataset('.././data/image/*.jpg','.././data/json/*.json')

[[BoundingBox(x1=85.1598, y1=23.7443, x2=218.4932, y2=79.2237, label=None),
  BoundingBox(x1=229.2237, y1=24.8858, x2=293.3790, y2=80.8219, label=None),
  BoundingBox(x1=11.6438, y1=103.6530, x2=77.8539, y2=130.8219, label=None),
  BoundingBox(x1=99.7717, y1=107.0776, x2=188.3562, y2=127.8539, label=None),
  BoundingBox(x1=197.9452, y1=108.6758, x2=218.0365, y2=128.9954, label=None),
  BoundingBox(x1=230.1370, y1=108.6758, x2=274.2009, y2=128.7671, label=None),
  BoundingBox(x1=1.8265, y1=157.9909, x2=122.1461, y2=185.3881, label=None),
  BoundingBox(x1=2.5114, y1=184.9315, x2=122.6027, y2=212.3288, label=None),
  BoundingBox(x1=2.2222, y1=212.1368, x2=124.2735, y2=238.1197, label=None),
  BoundingBox(x1=2.7350, y1=237.7778, x2=126.4957, y2=263.9316, label=None),
  BoundingBox(x1=3.5897, y1=264.4444, x2=126.4957, y2=291.2821, label=None),
  BoundingBox(x1=16.9231, y1=290.7692, x2=49.7436, y2=318.2906, label=None),
  BoundingBox(x1=145.4400, y1=159.8400, x2=252.8000, y2=186.5600, label=

In [4]:
import pandas as pd
import json
import glob
from tqdm import tqdm
images_path = '.././data/aug_img/'
json_path = '.././data/aug_json/'

images_list = glob.glob(images_path + '*.jpg')
json_list = glob.glob(json_path + '*.json')

images_list = [x.split('\\')[-1] for x in images_list]
json_list = [x.split('\\')[-1] for x in json_list]

images_list = sorted(images_list, key = lambda x : int(x.split('.')[0]))
json_list = sorted(json_list, key = lambda x : int(x.split('.')[0]))

ground_truth = []
for json_idx in tqdm(range(len(json_list))):
    with open("{}{}".format(json_path,json_list[json_idx]), "r") as file_json:
        file_json = json.load(file_json)
        for i,v in file_json.items():
            if i == 'type':
                continue
            for j in v:
                ground_truth.append([images_list[json_idx]])
                ground_truth[-1].append(json_list[json_idx])
                for key,value in j.items():
                    if type(value) == list:
                        for i in value:
                            ground_truth[-1].append(i) 
                    else: 
                        ground_truth[-1].append(value)
    
df = pd.DataFrame(ground_truth, columns =['Image_Name','Json_Name', 'Label','Xmin','Ymin','XMax','YMax']) 

  0%|          | 0/100000 [00:00<?, ?it/s]


IndexError: Replacement index 1 out of range for positional args tuple

In [33]:
ground_truth = []

for json_idx in tqdm(range(len(json_list))):
    with open("{}{}".format(json_path,json_list[json_idx]), "r") as file_json:
        file_json = json.load(file_json)
        for i,v in file_json.items():
            if i == 'type':
                continue
            for j in v:
                ground_truth.append([images_list[json_idx]])
                ground_truth[-1].append(json_list[json_idx])
                ground_truth[-1].append(j['label'])
                for q in j['points']:
                    ground_truth[-1].append(q)

100%|██████████| 100000/100000 [00:30<00:00, 3314.43it/s]


In [34]:
df = pd.DataFrame(ground_truth, columns =['Image_Name','Json_Name', 'Label','Xmin','Ymin','XMax','YMax']) 

In [35]:
len(df['Label'].unique().tolist())

56

In [36]:
df['Label']

0            고당
1           121
2             0
3          테이블명
4             5
           ... 
5735868       ,
5735869       ,
5735870       ,
5735871       ,
5735872       ,
Name: Label, Length: 5735873, dtype: object

In [32]:
ground_truth

[['0.jpg', '0.json', '고당', '786.9518', '2.3164434', '898.09656', '92.50901'],
 ['0.jpg', '0.json', '121', '616.3913', '1.1061777', '699.82745', '79.68898'],
 ['0.jpg', '0.json', '0', '722.04', '3.7577138', '764.8202', '78.247665'],
 ['0.jpg', '0.json', '테이블명', '201.12224', '167.59724', '435.9375', '320.1097'],
 ['0.jpg', '0.json', '5', '482.9746', '175.51147', '525.6033', '307.43527'],
 ['0.jpg', '0.json', '100', '693.38983', '318.98886', '774.767', '392.03174'],
 ['0.jpg',
  '0.json',
  '20210123',
  '14.885936',
  '312.6977',
  '269.38208',
  '389.92624'],
 ['0.jpg', '0.json', '17', '290.33698', '320.5671', '348.9399', '389.0118'],
 ['0.jpg', '0.json', '08', '369.94714', '318.17377', '438.6686', '388.16907'],
 ['0.jpg', '0.json', '42', '463.44534', '317.3579', '519.5423', '388.74258'],
 ['0.jpg',
  '0.json',
  '156',
  '491.89725',
  '400.38617',
  '575.81195',
  '474.92587'],
 ['0.jpg', '0.json', '영수번호', '150.6399', '402.4035', '378.6763', '489.79666'],
 ['0.jpg', '0.json', '단가', '3